In [ ]:
!pip install music21 pretty_midi

In [ ]:
from music21 import *
import numpy
import os
import pretty_midi
import glob
import pathlib
import pickle
import random
import multiprocessing
from multiprocessing import Pool
 
 '''
    process_file function takes a MIDI file and extracts all the data from it
    notes, chords, tempo, key, and time signature and returns a list of tuples
    containing all the information from the file.
    Args:
        midi_path (str):  Path to the MIDI file to be processed.
    Returns:
        list: A list of tuples containing note/chord, tempo, key, time signature, duration, and velocity information.
'''
def process_file(midi_path):
    # Parse the MIDI file
    print("Parsing %s" % midi_path)
    midi = converter.parse(midi_path)

    # Initialize lists to store extracted information
    notes = []
    tempos = []
    keys = []
    time_signatures = []

    # Extract tempo information
    try:
        bpm = midi.flat.getElementsByClass(tempo.TempoIndication)[0].getQuarterBPM()
        tempo_offset = midi.flat.getElementsByClass(tempo.TempoIndication)[0].offset
        tempos.append((bpm, tempo_offset))
    except:
        bpm = 120

    # Extract key information
    try:
        key = midi.analyze('key')
        keys.append(key)
    except:
        pass

    # Extract time signature information
    try:
        time_signature = midi.getTimeSignatures()[0]
        time_signature_offset = midi.flat.getElementsByClass(meter.TimeSignature)[0].offset
        time_signatures.append((time_signature.ratioString, time_signature_offset))
    except:
        pass

    # Extract note and chord data from either instrument parts or a flat structure
    notes_to_parse = None
    try:  # File has instrument parts
        s2 = instrument.partitionByInstrument(midi)
        notes_to_parse = s2.parts[0].recurse()
    except:  # File has notes in a flat structure
        notes_to_parse = midi.flat.notes

    # Iterate through notes/chords and extract relevant information
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            note_info = (str(element.pitch), str(bpm), str(key), str(time_signature))
            notes.append(note_info)
        elif isinstance(element, chord.Chord):
            chord_info = ('.'.join(str(n) for n in element.normalOrder), str(bpm), str(key), str(time_signature))
            notes.append(chord_info)

    # Return the list of notes and chords with extracted information
    return notes


'''
    get_midi_info gets all the notes, chords, tempo, key, and time signature from the MIDI files
    in the hand directory that's specified and uses multiprocessing to speed up the processing
    through calls to the process_file function.

    Args:
        hand_dir (str):   Name of the directory containing the MIDI files.
        parent_dir (str): Path to the parent directory containing the hand directory.

    Returns:
        list:   A list of tuples containing note/chord, tempo, key, time signature, duration, and velocity information.
'''
def get_midi_info(hand_dir, parent_dir):
    # Set the hand_name variable to the hand_dir input
    hand_name = hand_dir
    # Initialize an empty list to store notes
    notes = []

    # Create a new process pool using the 'with' statement
    with Pool() as p:
        # Iterate through each directory and sub-directory within the parent_dir
        for dirpath, _, filenames in os.walk(os.path.join(parent_dir, hand_dir)):
            # Iterate through each file in the current directory
            for file in filenames:
                # Check if the file has a '.mid' extension
                if file.endswith('.mid'):
                    # Create the full path to the MIDI file
                    midi_path = os.path.join(dirpath, file)
                    # Apply the process_file function asynchronously to the MIDI file and store the result
                    result = p.apply_async(process_file, args=(midi_path,))
                    # Get the result and extend the notes list with it
                    notes.extend(result.get())

        # Close the process pool after all tasks are done
        p.close()
        # Wait for all processes to finish before proceeding
        p.join()

    # Create a filename to save the extracted notes as a pickle file
    notes_filename = os.path.join(parent_dir, f'notes-{hand_name}.pkl')
    # Open the file in write-binary mode and dump the notes list using pickle
    with open(notes_filename, 'wb') as filepath:
        pickle.dump(notes, filepath)

    # Return the list of notes
    return notes

if __name__ == '__main__':
    part_dir = 'Midi_files'
    left_dir = 'Left'
    right_dir = 'Right'

    right_notes = get_midi_info(right_dir, part_dir)
    left_notes = get_midi_info(left_dir, part_dir)
